In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import (LinearRegression, 
                                  Ridge, 
                                  Lasso)
from sklearn.metrics import (r2_score, 
                             mean_absolute_error, 
                             mean_squared_error)

In [2]:
df = pd.read_csv('C:/Users/FAITH/Downloads/energydata_complete.csv')

In [3]:
del df['date']

In [9]:
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df.drop(columns=[ 'lights'])), 
                             columns=df.drop(columns=[ 'lights']).columns)
features_df = normalised_df.drop(columns=['Appliances'])
energy_target = normalised_df.Appliances
X_train, X_test, y_train, y_test = train_test_split(features_df, energy_target, test_size=.3, random_state=42)

In [10]:
lin_reg = LinearRegression()
lin_reg.fit(X_train[['T2']], X_train.T6)
T6_pred = lin_reg.predict(X_test[['T2']])
print(f'r^2 score: {round(r2_score(X_test.T6, T6_pred), 2)}')

r^2 score: 0.64


In [11]:
print(f'MAE: {round(mean_absolute_error(X_test.T6, T6_pred), 2)}')

MAE: 0.08


In [12]:
print(f'Residual Sum of Squares: {round(np.sum(np.square(X_test.T6 - T6_pred)), 2)}')

Residual Sum of Squares: 66.12


In [13]:
print(f'Root Mean Squared Error: {round(np.sqrt(mean_squared_error(X_test.T6, T6_pred)), 3)}')

Root Mean Squared Error: 0.106


In [16]:
df.drop(columns=['lights']).max().sort_values()

Windspeed        14.000000
Tdewpoint        15.500000
T9               24.500000
T5               25.795000
T7               26.000000
T_out            26.100000
T4               26.200000
T1               26.260000
T8               27.230000
T6               28.290000
T3               29.236000
T2               29.856667
rv1              49.996530
rv2              49.996530
RH_3             50.163333
RH_4             51.090000
RH_7             51.400000
RH_9             53.326667
RH_2             56.026667
RH_8             58.780000
RH_1             63.360000
Visibility       66.000000
RH_5             96.321667
RH_6             99.900000
RH_out          100.000000
Press_mm_hg     772.300000
Appliances     1080.000000
dtype: float64

In [18]:
df.drop(columns=['lights']).min().sort_values()

Tdewpoint       -6.600000
T6              -6.065000
T_out           -5.000000
Windspeed        0.000000
rv2              0.005322
rv1              0.005322
Visibility       1.000000
RH_6             1.000000
Appliances      10.000000
T9              14.890000
T4              15.100000
T5              15.330000
T7              15.390000
T2              16.100000
T8              16.306667
T1              16.790000
T3              17.200000
RH_2            20.463333
RH_7            23.200000
RH_out          24.000000
RH_1            27.023333
RH_4            27.660000
RH_3            28.766667
RH_9            29.166667
RH_8            29.600000
RH_5            29.815000
Press_mm_hg    729.300000
dtype: float64

In [19]:
def get_weights_df(model, feat, col_name):
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [20]:
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)

Ridge(alpha=0.4)

In [21]:
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

Lasso(alpha=0.001)

In [22]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [23]:
linear_model_weights = get_weights_df(model, X_train, 'Linear_Model_Weight')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weight')
lasso_weights_df = get_weights_df(lasso_reg, X_train, 'Lasso_weight')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights = pd.merge(final_weights, lasso_weights_df, on='Features')


In [24]:
final_weights.sort_values("Linear_Model_Weight", ascending=False)

,Features,Linear_Model_Weight,Ridge_Weight,Lasso_weight
25,rv1,4.860175e+11,0.000744,-0.000000
24,RH_1,5.533850e-01,0.519502,0.017880
23,T3,2.906708e-01,0.288055,0.000000
22,T6,2.364653e-01,0.217324,0.000000
21,Tdewpoint,1.178543e-01,0.083261,0.000000
20,T8,1.020108e-01,0.101028,0.000000
19,RH_3,9.620445e-02,0.095173,0.000000
18,RH_6,3.796818e-02,0.035500,-0.000000
17,Windspeed,2.912338e-02,0.030262,0.002912
16,T4,2.890953e-02,0.027364,-0.000000


In [25]:
y_pred_lg = model.predict(X_test)
y_pred_r = ridge_reg.predict(X_test)
y_pred_l = lasso_reg.predict(X_test)

In [26]:
print(f'Root Mean Squared Error: {round(np.sqrt(mean_squared_error(y_test, y_pred_r)), 3)}')

Root Mean Squared Error: 0.088


In [27]:
print(f'Root Mean Squared Error: {round(np.sqrt(mean_squared_error(y_test, y_pred_l)), 3)}')

Root Mean Squared Error: 0.094
